In [1]:
import numpy as np
import scipy.sparse as sps

In [2]:
# Parameters
n_inputs = 2
hidden_size = 1
n_hidden = 1

In [93]:
class Network(object): 
    def __init__(self, n_inputs, hidden_size, n_hidden_layers, learning_rate = 0.2):
        self.n_inputs = n_inputs
        self.input = []
        self.hidden_size = hidden_size
        self.n_hidden = n_hidden
        self.n_nodes = n_inputs + n_hidden*hidden_size + 1 #Number of nodes in network
        self.connections = np.array([[0 for x in range(self.n_nodes)] for y in range(self.n_nodes)], dtype = float )
        self.nodes = [0]*self.n_nodes
        self.lr = learning_rate
        
        # Initiate input nodes
        for i in range(n_inputs): 
            self.nodes[i] = Node(i)
            
        # Initiate hidden nodes
        #for i in range(n_hidden_layers): 
            
        # Initiate output node
        self.nodes[-1] = (Perceptron("SIGN"))
        
        # Set weights of connections
        for i in range(self.n_inputs): 
            self.connections[i][-1] = 1; # From input to perceptron: weight = 1. 
        

        print(self.connections)
            
    def test(self, inputs):
        # Takes input and returns output according to the network. 
        output = 0
        if len(inputs) != self.n_inputs:
            print("Wrong number of inputs %s, should be %s" %(inputs.length, self.n_inputs))
        
        for i, inp in enumerate(inputs):
            self.nodes[i].setValue(inp)
        
        return self.nodes[-1].output(self.connections, self.nodes)
    
    def fit(self, epochs, input_data, target_data):
        for epoch in range(epochs): 
            for i, row in enumerate(input_data):
                # get test(inputs)
                output = self.test(row)

                # if output == target: dont update weights
                if (output == target_data[i]):
                    pass
                else: # else: backpropagate. This needs to be updated to handle hidden layers in the future. 
                    for j, weight in enumerate(self.connections[:-1,2]):
                        # wi = wi_old + lr * self.nodes[i].output() * target
                        self.connections[j,2]= weight + self.lr * self.nodes[j].output() * target_data[i]
                        
                        print("Update w%s from %s to %s." %(j, weight, self.connections[j,2]))

                    weight = self.connections[-1,2]
                    self.connections[-1,2]= weight + self.lr * 1 * target_data[i] #Input in bias is always 1. 
                    #print("W: %s, T: %s, res: %s" %(weight, target_data[i], self.connections[-1,2]))
                    print("Updatee w%s from %s to %s." %(j+1, weight, self.connections[-1,2]))

                        
            print("Epoch %s out of %s" %(epoch+1, epochs))
            print(self.connections)


            # Fits the models according to the input and output data. 
            # Input is expected as an e * i matrix, output expected as e * 1

In [94]:
class Node(object):
    def __init__(self, index, value = None):
        self.value = value
        self.index = index 
        
    def output(self, connections = None, nodes = None): 
        # for each node in input
        return self.value  

    def setValue(self, value):
        self.value = value
    
class Perceptron(Node): 
    # Has a bunch of input-connections and output connections
    def __init__(self, value = None, activation_function = "SIGN"): 
        Node.__init__(self, value)
        self.activation_function = activation_function
        
    def sign(self, inp): 
        if inp < 0:
            return -1
        elif inp > 0: 
            return 1
        else: 
            return 0
        
    def output(self, connections, nodes):
        output = 0
        for i, weight in enumerate(connections[:-1,2]): # Recursive stuff
            output += nodes[i].output(connections, nodes)*weight
            
        
        output += 1*connections[-1][-1] # Add bias. 
        
        return self.sign(output)
        

In [95]:
n = Network(2,0,0,0.5)

# Train data for AND operator. 
input_data = [[0,0],[0,1],[1,0],[1,1]]
target_data = [-1,-1,-1,1]

# Fit the model. 
n.fit(5, input_data, target_data)



[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
Update w0 from 1.0 to 1.0.
Update w1 from 1.0 to 1.0.
Updatee w2 from 0.0 to -0.5.
Update w0 from 1.0 to 1.0.
Update w1 from 1.0 to 0.5.
Updatee w2 from -0.5 to -1.0.
Update w0 from 1.0 to 0.5.
Update w1 from 0.5 to 0.5.
Updatee w2 from -1.0 to -1.5.
Update w0 from 0.5 to 1.0.
Update w1 from 0.5 to 1.0.
Updatee w2 from -1.5 to -1.0.
Epoch 1 out of 5
[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0. -1.]]
Update w0 from 1.0 to 1.0.
Update w1 from 1.0 to 0.5.
Updatee w2 from -1.0 to -1.5.
Update w0 from 1.0 to 1.5.
Update w1 from 0.5 to 1.0.
Updatee w2 from -1.5 to -1.0.
Epoch 2 out of 5
[[ 0.   0.   1.5]
 [ 0.   0.   1. ]
 [ 0.   0.  -1. ]]
Update w0 from 1.5 to 1.5.
Update w1 from 1.0 to 0.5.
Updatee w2 from -1.0 to -1.5.
Update w0 from 1.5 to 1.0.
Update w1 from 0.5 to 0.5.
Updatee w2 from -1.5 to -2.0.
Update w0 from 1.0 to 1.5.
Update w1 from 0.5 to 1.0.
Updatee w2 from -2.0 to -1.5.
Epoch 3 out of 5
[[ 0.   0.   1.5]
 [ 0.   0.   1. ]
 [ 0.   0. 

In [92]:
# AND operator now working as expected
print(n.test([0,1]))
print(n.test([1,1]))

-1
1
